# Objetivo:

Criar código para que seja possível definir um ticket/empresa listada na bolsa e ser retornado todas as notícias relacionadas

as notícias devem conter:
* título
* data
* meio de comunicação
* conteúdo textual

In [5]:
# criar código para extrair o texto diretamente do link

Considerações 

    - ordenação por ordem de publicação
    - os links não funcionam
    - funciona melhor em ingles: data vem corretamente

## Bing news

### API Method

* METODO REQUEST

https://learn.microsoft.com/en-us/bing/search-apis/bing-news-search/reference/query-parameters


https://www.microsoft.com/en-us/bing/apis/pricing

In [5]:
import bing_search as bing
import pandas as pd
import util
from datetime import datetime 

In [6]:
def process_search(searches, result_count, period="Day"):
    dfs = []
    today = datetime.today().strftime('%Y-%m-%d')
    for search_term in searches:
        result = bing.search_news(search_term, result_count, period)
        result['full_text'] = result['url'].apply(util.extract_url_content)
        result['key'] = search_term
        result['data_de_busca'] = today
        dfs.append(result)
        
    dfs_result = pd.concat(dfs) 
    return dfs_result

In [7]:
searches = ['ITUB4', 'BBDC4', 'VALE3', 'MGLU3']
df_result = process_search(searches,5)
today = datetime.today().strftime('%Y-%m-%d')
df_result.to_parquet(f'result_{today}.parquet')

--------------

# blob storage 

In [1]:
import os, uuid
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [4]:
def blob_send_itens(local_file_name):
    try:
        print("Azure Blob Storage Python")

        account_url = "https://armazenamentonews.blob.core.windows.net"
        default_credential = DefaultAzureCredential()

        # Create the BlobServiceClient object
        blob_service_client = BlobServiceClient(account_url, credential=default_credential)

        # Create a file in the local data directory to upload and download
        container_name = 'arquivos'
        local_path = 'upload_file'
        # local_file_name = 'result_2024-01-13.parquet'
        upload_file_path = os.path.join(local_path, local_file_name)

        # Create a blob client using the local file name as the name for the blob
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=local_file_name)

        print("\nUploading to Azure Storage as blob:\n\t" + local_file_name)

        # Upload the created file
        with open(file=upload_file_path, mode="rb") as data:
            blob_client.upload_blob(data)
        return f'done:{local_file_name}'
    
    except Exception as ex:    
        result =  f'Exception:{ex}' 
        print(result)
        return result 

In [ ]:
item_sent = blob_send_itens('result_2024-01-13.parquet')